In [62]:
import os
import json
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
from collectors.etherscanapi import EtherscanAPI, get_timestamp_days_ago
from collectors.defillamaapi import DefillamaAPI

In [64]:
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
etherscan_api = EtherscanAPI(ETHERSCAN_API_KEY)

pd.set_option('display.max_columns', None)

## Retrieve Transaction Data for Fusion Settlement
0xA88800CD213dA5Ae406ce248380802BD53b47647 is the settlement contract address. We can use the contract address to retrieve the transaction data for the settlement contract.

In [65]:
timestamp = get_timestamp_days_ago(1)
block_number = etherscan_api.get_block_number_by_timestamp(timestamp)
block_number # block number to for start date of transaction query

'17048641'

In [66]:
# overwrite block number for testing:
block_number = 17055621

In [67]:
address = "0xA88800CD213dA5Ae406ce248380802BD53b47647"
transactions = etherscan_api.get_transactions(address, block_number)
df = etherscan_api.process_transactions(transactions)
df = df.query("isError == '0'")
df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName
0,17055621,1681601111,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,1843,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0,0x69313aec23db7e4e8788b942850202bcb6038734,0xa88800cd213da5ae406ce248380802bd53b47647,0,357894,101571620900,0,1,0x0965d04b000000000000000000000000000000000000...,,238596,238596,40,0x0965d04b,settleOrders(bytes data)
1,17055621,1681601111,0x59f8e728c18460ab5b5c49c109f09a34c7e6ece35dd1...,27009,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,2,0xc6c7565644ea1893ad29182f7b6961aab7edfed0,0xa88800cd213da5ae406ce248380802bd53b47647,0,1500000,32904071232,0,1,0x0965d04b000000000000000000000000000000000000...,,752164,245188,40,0x0965d04b,settleOrders(bytes data)


In [68]:
# each transaction has internal erc20 transfers that need to be extracted from the logs
# demo with just 1 for now:

receipt = etherscan_api.get_transaction_receipt(df.loc[0].hash)
logs = etherscan_api.process_logs(receipt)
logs

,address,data,blockNumber,transactionHash,transactionIndex,blockHash,logIndex,removed,from_address,to_address,transaction_type
0,0x514910771af9ca656af840dff83e8264ecf986ca,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x1,False,0xba9a1424a4508ef9184e5a0b5c37790336d76fd6,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,transfer
1,0x514910771af9ca656af840dff83e8264ecf986ca,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x2,False,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,0xaf0b0000f0210d0f421f0009c72406703b50506b,transfer
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x3,False,0xaf0b0000f0210d0f421f0009c72406703b50506b,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,transfer
3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x5,False,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,0xa88800cd213da5ae406ce248380802bd53b47647,transfer
4,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x6,False,0xa88800cd213da5ae406ce248380802bd53b47647,0x1111111254eeb25477b68fb85ed929f73a960582,None
5,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,0x1043f85,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,0x0,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0x7,False,0xa88800cd213da5ae406ce248380802bd53b47647,0xba9a1424a4508ef9184e5a0b5c37790336d76fd6,transfer


In [69]:
# get the unique token addresses involved in transfers
token_addresses = (
    logs
    .query("transaction_type == 'transfer'")
    ["address"]
    .unique()
)
token_addresses

array(['0x514910771af9ca656af840dff83e8264ecf986ca',
       '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], dtype=object)

In [70]:
defillama_api = DefillamaAPI()
token_data = defillama_api.get_historical_token_prices(token_addresses, df.loc[0].timeStamp)
token_data

ethereum:0x514910771af9ca656af840dff83e8264ecf986ca,ethereum:0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48


{'0x514910771af9ca656af840dff83e8264ecf986ca': {'decimals': 18,
  'symbol': 'LINK',
  'price': 8.04,
  'timestamp': 1681600969,
  'confidence': 0.99},
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'decimals': 6,
  'symbol': 'USDC',
  'price': 0.999863,
  'timestamp': 1681600968,
  'confidence': 0.99}}

In [71]:
# process the logs to get the erc20 transactions

def decode_data(data, transaction_type, decimals):
    if decimals is None or decimals is pd.NA:
        decimals = 18
    if transaction_type == "swap":
        # no longer working now that decimals will be nan...
        data = data[2:]
        return [int(data[i:i+64], 16) / 10 ** (decimals) for i in range(0, len(data), 64)]
    else: # assume transfer
        return int(data, 16) / 10 ** (decimals)

erc20_transfers = (
    logs
    .query("transaction_type == 'transfer'")
    .assign(
        decimals=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("decimals")),
        token_price=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("price")),
        symbol=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("symbol")),
    )
    .assign(
        data=lambda x: [decode_data(data, transaction_type, decimals) for data, transaction_type, decimals in zip(x["data"], x["transaction_type"], x["decimals"])]
    )
    .assign(
        usd_value=lambda x: [data * token_price for data, token_price in zip(x["data"], x["token_price"])]
    )
    .drop(columns=["blockHash", "blockNumber", "transactionHash", "removed", "logIndex", "transactionIndex", "address", "transaction_type", "decimals", ])
)

erc20_transfers

,data,from_address,to_address,token_price,symbol,usd_value
0,10005.000000,0xba9a1424a4508ef9184e5a0b5c37790336d76fd6,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,8.040000,LINK,80440.200000
1,10005.000000,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,0xaf0b0000f0210d0f421f0009c72406703b50506b,8.040000,LINK,80440.200000
2,80160.347595,0xaf0b0000f0210d0f421f0009c72406703b50506b,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,0.999863,USDC,80149.365627
3,80160.347594,0x358d4fe015cf721178d64799fdfdf37a31bbfe88,0xa88800cd213da5ae406ce248380802bd53b47647,0.999863,USDC,80149.365626
5,80160.347594,0xa88800cd213da5ae406ce248380802bd53b47647,0xba9a1424a4508ef9184e5a0b5c37790336d76fd6,0.999863,USDC,80149.365626


The first transfer informs us of the resolver's wallet address being used to execute the transaction (the to_address: 0x7cea8a6da079dc7501d4eb2bbc937e70a2764cef)
The outflow of the final token (FXS) to the settlement contract (0xa88800cd213da5ae406ce248380802bd53b47647) indicates the final token and the amount paid out.
The balance of the final token is the profit for the resolver.

In [72]:
# simplification and easiest measure of profit is just the difference in usd value of the first entry and the entry to settlement contract

initial_usd_value = erc20_transfers.iloc[0].usd_value
# filter for the transaction where to_address is 0xa88800cd213da5ae406ce248380802bd53b47647
final_usd_value = erc20_transfers.query("to_address == '0xa88800cd213da5ae406ce248380802bd53b47647'").usd_value.sum()
profit = initial_usd_value - final_usd_value # taken - given
profit

290.83437362036784

In [74]:
# getting profits for any transaction:


receipts = [etherscan_api.get_transaction_receipt(hash) for hash in df.hash]
logs = [etherscan_api.process_logs(receipt) for receipt in receipts]

profit_volume_list = []

for log, timestamp in zip(logs, df.timeStamp):
    # get the unique token addresses involved in transfers
    token_addresses = (
        log
        .query("transaction_type == 'transfer'")
        ["address"]
        .unique()
    )
    # get the token data for the tokens involved in the transaction
    token_data = defillama_api.get_historical_token_prices(token_addresses, timestamp)
    # process the logs to get the erc20 transactions
    erc20_transfers = (
        log
        .query("transaction_type == 'transfer'")
        .assign(
            decimals=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("decimals")),
            token_price=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("price")),
            symbol=lambda x: x["address"].map(lambda address: token_data.get(address, {}).get("symbol")),
        )
        .assign(
            data=lambda x: [decode_data(data, transaction_type, decimals) for data, transaction_type, decimals in zip(x["data"], x["transaction_type"], x["decimals"])]
        )
        .assign(
            usd_value=lambda x: [data * token_price for data, token_price in zip(x["data"], x["token_price"])]
        )
        # .drop(columns=["blockHash", "blockNumber", "transactionHash", "removed", "logIndex", "transactionIndex", "address", "transaction_type", "decimals", ])
    )
    # measure profit (and volume)
    initial_usd_value = erc20_transfers.iloc[0].usd_value
    # filter for the transaction where to_address is 0xa88800cd213da5ae406ce248380802bd53b47647
    final_usd_value = erc20_transfers.query("to_address == '0xa88800cd213da5ae406ce248380802bd53b47647'").usd_value.sum()
    profit = initial_usd_value - final_usd_value # taken - given

    # "true volume" is the sum of only the initial and final; the others are intermediate.
    true_volume = initial_usd_value + final_usd_value

    profit_volume_list.append({"profit": profit, "true_volume": true_volume})

# append to df
df = df.assign(**pd.DataFrame(profit_volume_list))
df.head()
                    

ethereum:0x514910771af9ca656af840dff83e8264ecf986ca,ethereum:0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
ethereum:0x45c2f8c9b4c0bdc76200448cc26c48ab6ffef83f,ethereum:0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,profit,true_volume
0,17055621,1681601111,0x626b2e3ecb31f95b9ebefd45c3bbd057b28d4af68197...,1843,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,0,0x69313aec23db7e4e8788b942850202bcb6038734,0xa88800cd213da5ae406ce248380802bd53b47647,0,357894,101571620900,0,1,0x0965d04b000000000000000000000000000000000000...,,238596,238596,40,0x0965d04b,settleOrders(bytes data),290.834374,160589.565626
1,17055621,1681601111,0x59f8e728c18460ab5b5c49c109f09a34c7e6ece35dd1...,27009,0xe37dd7a224c7a12bb0e7d007bc4265b5342410556843...,2,0xc6c7565644ea1893ad29182f7b6961aab7edfed0,0xa88800cd213da5ae406ce248380802bd53b47647,0,1500000,32904071232,0,1,0x0965d04b000000000000000000000000000000000000...,,752164,245188,40,0x0965d04b,settleOrders(bytes data),-3.006744,293.500644


In [77]:
resolver_address_mapping ={
    '0x55dcad916750c19c4ec69d65ff0317767b36ce90': '1inch Labs',
    '0x8acdb3bcc5101b1ba8a5070f003a77a2da376fe8': '1inch Labs',
    '0x84d99aa569d93a9ca187d83734c8c4a519c4e9b1': '1inch Labs',
    '0x3169de0e661d684e0d235f19cf72327173e0be11': '1inch Labs',
    '0x9108813f22637385228a1c621c1904bbbc50dc25': 'Laertes',
    '0x2eb393fbac8aaa16047d4242033a25486e14f345': 'Arctic Bastion',
    '0x7636a5bfd763cefec2da9858c459f2a9b0fe8a6c': 'Arctic Bastion',
    '0xf1b2e1fef70e0383ef29618d02d0dd503ae239ae': 'Arctic Bastion',
    '0x377a1286ff83df266ff11bede2ef600044f3626b': 'Arctic Bastion',
    '0xcfa62f77920d6383be12c91c71bd403599e1116f': 'The Open DAO resolver',
    '0xad7149152a65e6ec97add7b1b1f917dcafcf9b21': 'Seawise',
    '0xd1742b3c4fbb096990c8950fa635aec75b30781a': 'Seawise',
    '0xc975671642534f407ebdcaef2428d355ede16a2c': 'OTEX',
    '0xbd4dbe0cb9136ffb4955ede88ebd5e92222ad09a': 'OTEX',
    '0xc6c7565644ea1893ad29182f7b6961aab7edfed0': 'The T Resolver',
    '0x69313aec23db7e4e8788b942850202bcb6038734': 'Resolver 8',
    '0xee230dd7519bc5d0c9899e8704ffdc80560e8509': 'Kinetex Labs Resolver'
}


In [78]:
# groupby resolvers
(
    df
    .assign(
        transactionCost=lambda x: x["gasUsed"] * x["gasPrice"] / 1e18,
        **{"from": lambda x: x["from"].map(lambda address: resolver_address_mapping.get(address, address))},
    )
    .groupby("from")
    .agg(
        total_gas_used=("gasUsed", "sum"),
        avg_gas_used=("gasUsed", "mean"),
        avg_gas_price=("gasPrice", "mean"),
        total_transaction_cost=("transactionCost", "sum"),
        avg_transaction_cost=("transactionCost", "mean"),
        total_profit=("profit", "sum"),
        avg_profit=("profit", "mean"),
        total_volume=("true_volume", "sum"),
        avg_volume=("true_volume", "mean"),
    )
)

,total_gas_used,avg_gas_used,avg_gas_price,total_transaction_cost,avg_transaction_cost,total_profit,avg_profit,total_volume,avg_volume
from,,,,,,,,,
Resolver 8,238596,238596.0,1.015716e+11,0.024235,0.024235,290.834374,290.834374,160589.565626,160589.565626
The T Resolver,245188,245188.0,3.290407e+10,0.008068,0.008068,-3.006744,-3.006744,293.500644,293.500644
